<a href="https://colab.research.google.com/github/ArySuby/sasap-test/blob/main/AutosMdPMercadoLibre.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [55]:
!pip install beautifulsoup4


In [56]:
!pip install playwright
!playwright install chromium


In [70]:
import asyncio
import nest_asyncio
from playwright.async_api import async_playwright
import pandas as pd
import requests

nest_asyncio.apply()

# 🔹 Obtener dólar oficial
def obtener_dolar_oficial():
    try:
        url = "https://api.bluelytics.com.ar/v2/latest"
        resp = requests.get(url, timeout=10)
        data = resp.json()
        oficial = data.get("oficial") or data.get("official")
        if oficial and "value_sell" in oficial:
            return float(oficial["value_sell"])
    except Exception as e:
        print(f"Error al obtener dólar oficial Bluelytics: {e}")
    return None

# 🔹 Scraping paginado por _Desde_N
async def scrapear_autos():
    base_url = "https://autos.mercadolibre.com.ar/bsas-costa-atlantica/mar-del-plata/usados/_Desde_{}"
    autos = []
    offset = 0
    page_size = 48

    async with async_playwright() as pw:
        browser = await pw.chromium.launch(headless=True)
        page = await browser.new_page()

        while True:
            print(f"Scrapeando página con offset {offset}...")
            url = base_url.format(offset)
            await page.goto(url, wait_until='domcontentloaded')
            await page.wait_for_timeout(3000)

            items = await page.query_selector_all("li.ui-search-layout__item")
            if not items:
                break  # fin de resultados

            for el in items:
                try:
                    title = await el.eval_on_selector("h2, h3", "e => e.textContent.trim()")
                    num = await el.eval_on_selector("span.andes-money-amount__fraction", "e => e.textContent.trim()")
                    sym = await el.eval_on_selector("span.andes-money-amount__currency-symbol", "e => e.textContent.trim()")
                except:
                    continue

                attrs = await el.query_selector_all("ul.poly-attributes_list li")
                year = km = None
                for at in attrs:
                    txt = (await at.inner_text()).strip()
                    if txt.isdigit() and len(txt) == 4:
                        year = txt
                    elif "km" in txt.lower():
                        km = txt

                marca = title.split()[0]
                modelo = " ".join(title.split()[1:])

                autos.append({
                    "marca": marca,
                    "modelo": modelo,
                    "año": year,
                    "kilometraje": km,
                    "precio_orig": int(num.replace(".", "")),
                    "moneda": sym
                })

            offset += page_size  # siguiente página

        await browser.close()
    return autos

# 🔹 Ejecutar todo
autos = asyncio.get_event_loop().run_until_complete(scrapear_autos())

dolar_oficial = obtener_dolar_oficial()
print(f"Dólar oficial (venta, BNA vía Bluelytics): {dolar_oficial}")

# Normalizar precios
for auto in autos:
    if auto["moneda"] == "US$":
        auto["precio_usd"] = auto["precio_orig"]
    elif auto["moneda"] == "$" and dolar_oficial:
        auto["precio_usd"] = round(auto["precio_orig"] / dolar_oficial, 2)
    else:
        auto["precio_usd"] = None

# 🔹 Guardar CSV
df = pd.DataFrame(autos)
df.to_csv("autos_ml_mar_del_plata_usd.csv", index=False)
print(f"✅ Total autos scrapeados: {len(df)}")
print(df.head())


Scrapeando página con offset 0...
Scrapeando página con offset 48...
Scrapeando página con offset 96...
Scrapeando página con offset 144...
Scrapeando página con offset 192...
Scrapeando página con offset 240...
Scrapeando página con offset 288...
Scrapeando página con offset 336...
Scrapeando página con offset 384...
Scrapeando página con offset 432...
Scrapeando página con offset 480...
Scrapeando página con offset 528...
Scrapeando página con offset 576...
Scrapeando página con offset 624...
Scrapeando página con offset 672...
Scrapeando página con offset 720...
Scrapeando página con offset 768...
Scrapeando página con offset 816...
Scrapeando página con offset 864...
Scrapeando página con offset 912...
Scrapeando página con offset 960...
Scrapeando página con offset 1008...
Scrapeando página con offset 1056...
Scrapeando página con offset 1104...
Scrapeando página con offset 1152...
Scrapeando página con offset 1200...
Scrapeando página con offset 1248...
Scrapeando página con offs

In [75]:
import pandas as pd

# Leer el CSV limpio previo
df = pd.read_csv("autos_ml_mar_del_plata_clean.csv")

# Convertir columnas a tipo adecuado
df["año"] = pd.to_numeric(df["año"], errors="coerce")

# Extraer números del kilometraje y convertir a int
df["kilometraje"] = df["kilometraje"].str.replace(".", "", regex=False)  # sacamos puntos
df["kilometraje"] = pd.to_numeric(df["kilometraje"].str.extract(r"(\d+)")[0], errors="coerce")

# Convertir precio_usd a num (por si acaso)
df["precio_usd"] = pd.to_numeric(df["precio_usd"], errors="coerce")

# Aplicar filtros
df_filtrado = df[
    (df["año"] >= 2010) &
    (df["año"] <= 2025) &
    (df["kilometraje"] > 0) &
    (df["precio_usd"] >= 3000) &
    (df["precio_usd"] <= 60000)
].copy()

# Guardar nuevo CSV
df_filtrado.to_csv("autos_ml_mar_del_plata_usd_filtrado.csv", index=False)

print(f"Autos después de filtrar: {len(df_filtrado)}")
print(df_filtrado.head())


Autos después de filtrar: 1788
           marca                                     modelo     año  \
0        Renault                   Captur 2.0 Intens Manual  2017.0   
1  Mercedes-benz  Clase Glk 3.0 Glk300 4matic City 231cv At  2013.0   
2         Toyota                     Etios 1.5 Sedan Xls At  2019.0   
3  Mercedes-benz             Clase Glc Glc 300 Coupé 4matic  2017.0   
4           Ford                  Focus Iii 2.0 Se Plus At6  2015.0   

   kilometraje  precio_orig moneda  precio_usd  kilometraje_num  
0        80000     21000000      $    17241.38            80000  
1       155000        19900    US$    19900.00           155000  
2        90000     20000000      $    16420.36            90000  
3        80000        59900    US$    59900.00            80000  
4        90000     17300000      $    14203.61            90000  
